# BC 5
## Flat ensemble of tiny networks
## Input = sub-region

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
SEED = 321
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
NUM_TN = 30
SUB_REGION_SCALE = 14

#dataset specific parameters
NUM_CLASS = 5

In [4]:
data = "../input/flowers-recognition/flowers/"
folders = os.listdir(data)

In [5]:
image_names = []
train_labels = []
train_images = []

size = 32,32

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

In [6]:
train = np.array(train_images)
train = train.astype('float32')
train.shape

(4323, 32, 32, 3)

In [7]:
# Train, Val, Test split = 0.8, 0.1, 0.1 of the dataset
X_train,X_val,y_train,y_val = train_test_split(train,train_labels, test_size = 0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val, test_size = 0.5)

## Data Pre-Processing

In [8]:
#Generate cropped train image
X_train_cropped_list = []
y_train_cropped_list = []

for i in range(NUM_TN):
  X_train_cropped = []
  y_train_cropped = []
  for i in range (X_train.shape[0]):
    image_cropped = tf.image.random_crop(X_train[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
    image_cropped = np.array(image_cropped)
    X_train_cropped.append(image_cropped)
    y_train_cropped.append(y_train[i])
  X_train_cropped = np.array(X_train_cropped)
  y_train_cropped = np.array(y_train_cropped)

  X_train_cropped_list.append(X_train_cropped)
  y_train_cropped_list.append(y_train_cropped)

In [9]:
#Generate cropped val image
X_val_cropped_list = []
y_val_cropped_list = []

for i in range(NUM_TN):
    X_val_cropped = []
    y_val_cropped = []
    for i in range (X_val.shape[0]):
        image_cropped = tf.image.random_crop(X_val[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
        image_cropped = np.array(image_cropped)
        X_val_cropped.append(image_cropped)
        y_val_cropped.append(y_val[i])
    X_val_cropped = np.array(X_val_cropped)
    y_val_cropped = np.array(y_val_cropped)

    X_val_cropped_list.append(X_val_cropped)
    y_val_cropped_list.append(y_val_cropped)

In [10]:
#Generate cropped test image
X_test_cropped_list = []
y_test_cropped_list = []

for i in range(NUM_TN):
    X_test_cropped = []
    y_test_cropped = []
    for i in range (X_test.shape[0]):
        image_cropped = tf.image.random_crop(X_test[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
        image_cropped = np.array(image_cropped)
        X_test_cropped.append(image_cropped)
        y_test_cropped.append(y_test[i])
    X_test_cropped = np.array(X_test_cropped)
    y_test_cropped = np.array(y_test_cropped)

    X_test_cropped_list.append(X_test_cropped)
    y_test_cropped_list.append(y_test_cropped)

In [11]:
#normalizing dataset
for i in range(NUM_TN):
    X_train_cropped_list[i] = X_train_cropped_list[i]/255
    X_val_cropped_list[i] = X_val_cropped_list[i]/255
    X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [12]:
#transform to vector
for i in range(NUM_TN):
    X_train_cropped_list[i] = X_train_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))
    X_val_cropped_list[i] = X_val_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))
    X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

In [13]:
#one-hot encoding
for i in range(NUM_TN):
    y_train_cropped_list[i] = pd.get_dummies(y_train_cropped_list[i])
    y_train_cropped_list[i] = y_train_cropped_list[i].values.argmax(1)
    y_train_cropped_list[i] = to_categorical(y_train_cropped_list[i])
    
    y_val_cropped_list[i] = pd.get_dummies(y_val_cropped_list[i])
    y_val_cropped_list[i] = y_val_cropped_list[i].values.argmax(1)
    y_val_cropped_list[i] = to_categorical(y_val_cropped_list[i])
    
    y_test_cropped_list[i] = pd.get_dummies(y_test_cropped_list[i])
    y_test_cropped_list[i] = y_test_cropped_list[i].values.argmax(1)
    y_test_cropped_list[i] = to_categorical(y_test_cropped_list[i])

## Model

In [14]:
#build ANN model
ensemble = []
for i in range(NUM_TN):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append(model)

In [15]:
#train model
history = []
for i in range(NUM_TN):
    print()
    print('Model %d' %i)
    print()
    hist = ensemble[i].fit(X_train_cropped_list[i], y_train_cropped_list[i], validation_data=(X_val_cropped_list[i], y_val_cropped_list[i]), epochs=5, batch_size=128)
    history.append(hist)


Model 0

Epoch 1/5
28/28 [==============================] - 2s 21ms/step - loss: 1.5896 - accuracy: 0.2732 - val_loss: 1.4724 - val_accuracy: 0.3171
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.4540 - accuracy: 0.3487 - val_loss: 1.3989 - val_accuracy: 0.3657
Epoch 3/5
28/28 [==============================] - 0s 4ms/step - loss: 1.4115 - accuracy: 0.3686 - val_loss: 1.3824 - val_accuracy: 0.3681
Epoch 4/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3849 - accuracy: 0.3855 - val_loss: 1.3755 - val_accuracy: 0.3935
Epoch 5/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3742 - accuracy: 0.3833 - val_loss: 1.3778 - val_accuracy: 0.3657

Model 1

Epoch 1/5
28/28 [==============================] - 1s 9ms/step - loss: 1.5909 - accuracy: 0.2483 - val_loss: 1.4592 - val_accuracy: 0.3634
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.4339 - accuracy: 0.3569 - val_loss: 1.4064 - val_accuracy: 0.3472
Epoch 

## Model Evaluation

In [16]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN):
    y_pred = ensemble[i].predict(X_test_cropped_list[i])
    y_pred_list.append(y_pred)
    
# Ensemble voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.argmax(np.sum(y_pred_list, axis=0), axis=1)

In [17]:
y_test_label_encoded_list =[]

In [18]:
for k in y_test_cropped_list:
    y_test_label_encoded_list.append(k)
    
y_test_label_encoded_list  = np.array(y_test_label_encoded_list)
y_test_label_encoded_list = np.argmax(np.sum(y_test_label_encoded_list, axis=0), axis=1)

In [19]:
accuracy = accuracy_score(y_test_label_encoded_list, y_pred_list)
accuracy

0.4341801385681293